# Packages 

In [ ]:
# !pip install pandas
# !pip install pandarallel
# !pip install unidecode

In [ ]:
import json
import pandas as pd
from pandas import DataFrame
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import glob
import pathlib
import unicodedata
import tarfile

from pandarallel import pandarallel
pandarallel.initialize()
from unidecode import unidecode

t0 = time.time()

def get_csv_size(csv_name):
    get_size = os.path.getsize(os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024 * 1024)
    mb_size = round(mb_size,1)
    return mb_size

def get_attribute(filename):
    if filename.endswith('csv'):
        df1 = pd.read_csv(filename,header=[0])
    else:
        df1=pd.read_json(filename,orient="records", lines=True)
    return list(df1.columns) 

def split_large_file(filename, source, output,size=None):
    df1=pd.read_csv(filename,header=[0])
    if size ==None:
        size = get_csv_size(filename)
    num_chunks = size//20
    if num_chunks == 0:
        num_chunks = 1 
    df_all = np.array_split(df1, num_chunks)

    for idx, file in enumerate(df_all):
        file.to_csv('%s/%s_%03d.csv'%(output, source, idx), index=False)   
        
def show_all_files(folder):
    df = pd.DataFrame(glob.glob('%s/*'%folder), columns = ['path'])
    df['root'] = df.path.apply(lambda x: x.split('/')[0])
    df['source'] = df.path.apply(lambda x: x.replace('%s/'%folder,'').split('_')[0])
    df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,'').split('/')[-1])
    df = df.sort_values('source').reset_index(drop=True)
    df['size'] = df['path'].apply(get_csv_size)
    return df

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

# Check csv

In [ ]:
pandarallel.initialize(nb_workers=94, progress_bar=True) 
# Place all files to be filtered within the Filter folder, within a subfolder containing the source name
# for example, let there be 3 csv files from El Espectador. 
# The file structure is 
# Filter
# --elespectador
# -- -- elespactador01.csv
# -- -- elespactador02.csv
# -- -- elespactador03.csv


# This creates a dataframe containing a list of all the csv files within the specified folder structure.
df = pd.DataFrame(glob.glob('*/*.csv'), columns = ['path'])
#print(df)
# This updates the source to be just the first part, the name of the folder.
df['source'] = df.path.apply(lambda x: x.split('/')[1].replace('.csv', ''))
df = df.sort_values('source').reset_index(drop=True)
# Gets the column defenitions of the files and their sizes
df['attributes']= df.path.apply(get_attribute)
df['size']= df.path.apply(get_csv_size)
df

Remove unrequired columns like 'Unnamed: 0' and 'Unnamed: 1'

In [ ]:
 for p in df.path:
     current_df = pd.read_csv(p, index_col=0)
     if 'Unnamed: 0' in current_df.columns:
         current_df = current_df.drop(['Unnamed: 0'], axis=1)
        
     if 'Unnamed: 1' in current_df.columns:
         current_df = current_df.drop(['Unnamed: 1'], axis=1)
        
     if 'Unnamed: 0.1' in current_df.columns:
         current_df = current_df.drop(['Unnamed: 0.1'], axis=1)
     current_df.to_csv(p)

In [ ]:
# df = df[df.attributes.map(len)==6] #Only get the CSVs which have the appropriate number of columns
# df

In [ ]:
df['size'].sum()

In [ ]:
# df_another = pd.DataFrame(glob.glob('split/*_filtered.csv'), columns = ['path'])
# df_another['size']= df_another.path.apply(get_csv_size)
# df_another['size'].sum()

Initialize filter files.

In [ ]:
import re


class Trie():
    """Regex::Trie in Python. Creates a Trie out of a list of words. The trie can be exported to a Regex pattern.
    The corresponding Regex should match much faster than a simple Regex union."""

    def __init__(self):
        self.data = {}

    def add(self, word):
        ref = self.data
        for char in word:
            ref[char] = char in ref and ref[char] or {}
            ref = ref[char]
        ref[''] = 1

    def dump(self):
        return self.data

    def quote(self, char):
        return re.escape(char)

    def _pattern(self, pData):
        data = pData
        if "" in data and len(data.keys()) == 1:
            return None

        alt = []
        cc = []
        q = 0
        for char in sorted(data.keys()):
            if isinstance(data[char], dict):
                try:
                    recurse = self._pattern(data[char])
                    alt.append(self.quote(char) + recurse)
                except:
                    cc.append(self.quote(char))
            else:
                q = 1
        cconly = not len(alt) > 0

        if len(cc) > 0:
            if len(cc) == 1:
                alt.append(cc[0])
            else:
                alt.append('[' + ''.join(cc) + ']')

        if len(alt) == 1:
            result = alt[0]
        else:
            result = "(?:" + "|".join(alt) + ")"

        if q:
            if cconly:
                result += "?"
            else:
                result = "(?:%s)?" % result
        return result

    def pattern(self):
        return self._pattern(self.dump())

def trie_regex_from_words(words):
    trie = Trie()
    for word in words:
        trie.add(word)
    return re.compile(r"\b" + trie.pattern() + r"\b", re.IGNORECASE)

In [ ]:
# Initialize the relevant keywords and the irrelevant keyword regex's
with open('wiki_relevant_arabic_v2.txt', 'r', encoding='utf8') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

# relevant_rules = re.compile(relevant_rules, re.IGNORECASE)
print(relevant_rules)

In [ ]:
# Initialize the relevant keywords and the irrelevant keyword regex's
with open('wiki_relevant_arabic_v2.txt', 'r', encoding='utf8') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irrelevant_keywords_arabic_v2.txt', 'r', encoding='utf8') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)

# debugging, no longer relevant
#find_relevant = lambda x: Counter(relevant_rules.findall(str(x).lower()))
#find_exclude = lambda x: Counter(irelevant_rules.findall(str(x).lower()))

In [ ]:
# remove all the filtered paths, if they exist. Debugging
# df['path'] = df['path'].apply(lambda x: x[:-4] + '_filtered.csv')

In [ ]:
# Checks for doublely filtered files. Debugging
df = df[~df.path.str.contains('_filtered_filtered')]

# Split

In [ ]:
df

In [ ]:
# Makes directory for splits. Do not run if a split folder already exists.
cwd = os.getcwd()
#cwd = '/xdisk/josorio1/salsarra/Filtered/'
os.mkdir(cwd + '/split/')
cwd

In [ ]:
# splits all files above a certain MB limit
pandarallel.initialize(nb_workers=94, progress_bar=True)       
_ = df.apply(lambda x: split_large_file(x['path'],x['source'],'split'), axis=1) 

In [ ]:
# Debugging, if there are issues here, let me know.
df = show_all_files('split')
df = df[~df.path.str.contains('_filtered')]
df

In [ ]:
# Check file sizes.
df['size'].describe()

# step 1  clean

In [ ]:
df = show_all_files('split')
# df['attribute'] = df.path.parallel_apply(get_attribute)
df

In [ ]:
df.source.unique()

In [ ]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

In [ ]:
# Cleans the given files.
def basic_process(filename, output_folder):
    
    sizes = []
    
    df1 = pd.read_csv(filename, header=[0], index_col=0)
    sizes.append(df1.shape[0])
    
    if 'url' not in df1.columns:
        df1['url'] ='\n'
    if 'title' not in df1.columns:
        df1['title']='\n'
    
#     df1 = df1[~df1.url.isnull()]
    
    df1 = df1[~df1.text.isnull()]
    df1 = df1[~df1.text.duplicated()]
#     df1 = df1[~df1.url.duplicated()]
    
    df1.loc[df1.title.isnull(),'title']='\n'
    df1.loc[df1.url.isnull(),'url']='\n'
    if 'abstract' in df1:
        df1.loc[df1.abstract.isnull(),'abstract']='\n'
        
    # ------------------- Start cleaning  --------------------------#
    
    # Convert coding
    df1.text = df1.text.apply(lambda x: unicodetoascii(x))
    df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))
    
    # email
    df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
    
    # telphone
    df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
    df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))
    
    # noise
    df1.text =\
    df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))
    
    # urls
    df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))
    
    # delete too many \n
    df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))
    
    # head and tails
    df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 
    
    
    df1 = df1[df1.text.str.len()>100]
    
    sizes.append(df1.shape[0])
    
    # ------------------- Ending cleaning  --------------------------#
    
    filename = filename.split('/')[1]
    new_filename = output_folder + '/'+ filename

    print('%s:\t%s'%(filename, sizes))
    df1.to_csv(new_filename, index= False)
    return

In [ ]:
# Takes the splits and cleans them.
pandarallel.initialize(nb_workers=94, progress_bar=False) 
_ = df['path'].apply(basic_process, output_folder='split')

In [ ]:
df = show_all_files('split')
df = df[~df.path.str.contains('filtered')]
df

In [ ]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

### Filter

In [ ]:
# defines a metric for number of relevant and irrelevant words.
def find_relevant(str):
    return len(relevant_rules.findall(str))/len(str.split())
def find_irelevant(str):
    return len(irelevant_rules.findall(str))/len(str.split())

In [ ]:
# find_relevant = lambda x: Counter(relevant_rules.findall(x.lower()))
# find_exclude = lambda x: Counter(irelevant_rules.findall(x.lower()))
#

# for each file, open and read into dataframe, then make sure text is NOT null
def filter(filepath, filter_for_positive=True):
    print(f'Filter for positive is {filter_for_positive}')
    sizes = []
    df1 = pd.read_csv(filepath, header=[0], index_col=0)
    if 'text' not in df1.columns:
        print(f'{filepath} has no text column')
        return 
    sizes.append(df1.shape[0]) 
    print(filepath)
    print(len(df1))
    print(df1.columns)
    df1 = df1[~df1.text.isnull()]
    print(len(df1))
#     df1 = df1[~df1['text'] == '']
    print(len(df1))
    df1['relevant'] = df1['text'].apply(find_relevant)
    df1['irrelevant'] = df1['text'].apply(find_irelevant)

    # df1['ratio'] = df1['relevant']/df1['irrelevant']
    relevant_count = df1.relevant
    irrelevant_count = df1.irrelevant
    

    # ratio_values = df1.ratio
# Filter Calculation: Mean and SD of both relevant and irrelevant columns
    relevant_mean = np.mean(df1['relevant'])
    irrelevant_mean = np.mean(df1['irrelevant'])
    relevant_sd = np.std(df1['relevant'])
    irrelevant_sd = np.std(df1['irrelevant'])
    # print(np.mean(df1['ratio']), np.std(df1['ratio']))
    # print(df1['relevant'].describe())
    print(relevant_mean, relevant_sd)
    # print(df1['irrelevant'].describe())
    print(irrelevant_mean, irrelevant_sd) 
# The next three lines are the filter, can be changed based on the needs of the project. Currently, throw out all rows with more than two irrelevant words/regex
    threshold = 1.5
    # filter based on ratio rather than just counts    
    select_index = (relevant_count >= irrelevant_count) & (irrelevant_count < irrelevant_mean + threshold * irrelevant_sd) & (relevant_count > relevant_mean - threshold * relevant_sd) # (ratio_values > np.mean(df1['ratio']) - threshold * np.std(df1['ratio']))
    select_index = [x if filter_for_positive else not x for x in select_index]

    df1 = df1[select_index]   
    df1 = df1.drop('relevant', axis=1)
    df1 = df1.drop('irrelevant', axis=1)
    # df1 = df1.drop('ratio', axis=1)
    sizes.append(df1.shape[0])
# next line is debugging    
    # print(df1)
# create a filtered file with the given dataframe, will appear in same folder as unfiltered
    
    if filter_for_positive:  
        df1.to_csv(filepath[:-4] + '_filtered.csv', index=False)
    else:
        df1.to_csv(filepath[:-4] + '_irrelevant_filtered.csv', index=False)
    print('%s:\t%s'%(filepath[6:-4], sizes))

In [ ]:
# filter("split/Guatemala_000.csv")
# df_x = pd.read_csv("split/Guatemala_000.csv", header=[0], index_col=0)
# # df_x.iloc[:, 0]
# # df_x.columns
# df_x[[:, 'url']]

In [ ]:
df

In [ ]:
pandarallel.initialize(nb_workers=94, progress_bar=True) 
_ = df['path'].parallel_apply(filter, filter_for_positive=True) #Get relevant articles

_ = df['path'].parallel_apply(filter, filter_for_positive=False) #Get irrelevant articles

# save to json

In [ ]:
# folder = 'none'
# df = pd.DataFrame(columns=['path','source', 'filename'])
# df.path = [str(x) for x in pathlib.Path('%s/*'%folder).glob('**/*')]
df = pd.DataFrame(glob.glob('*/*'), columns = ['path'])
df = df[df.path.str.contains('_filtered')]
# df['root'] = df.path.apply(lambda x: x.split('/')[0])

df['source'] = df.path.apply(lambda x: x.split('/')[1][0:x.index('_')-6])
df['filename'] = df.path.apply(lambda x: x.split('/')[-1])

# df = df.sort_values('source').reset_index(drop=True)
# df['size'] = df['path'].parallel_apply(get_csv_size)
# df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,''))

df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))

df['json_file'] = df.source +'/'+ df.json_file
df['tar_file'] = 'tar/'+ df.json_file+'.tar.gz'
df.json_file = 'json/'+ df.json_file
df

In [ ]:
os.mkdir(cwd + '/json/')
os.mkdir(cwd + '/tar/')

for i in df.source.unique():
    os.mkdir(cwd + '/json/'+i)
    os.mkdir(cwd + '/tar/'+i)  

In [ ]:
df1 = pd.read_csv(df.path[1], header=[0], index_col=0)
df1

In [ ]:
#----------------------this was commented out-----------
# def convert_json_tar(filename, json_file, tar_file):
#     df1 = pd.read_csv(filename, header=[0], index_col=0)      
#     df1.to_json(json_file, orient="records", lines=True)
    
#     with tarfile.open(tar_file, "w:gz") as tar:
#         tar.add(json_file, arcname=os.path.basename(json_file))
# df.apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

In [ ]:
pandarallel.initialize() 

df.source = df.path.apply(lambda x: x.split('/')[1][0:x.index('_')-6]).values
df['size'] = df['path'].apply(get_csv_size)
df

In [ ]:
for filename in next(os.walk("split"), (_, _, []))[2]:
    if filename.endswith("irrelevant_filtered.csv"):
        print(filename)

In [ ]:
print(df['size'].sum())

df.groupby(by=["path"], dropna=False).sum()

In [ ]:
l = []
for filename in next(os.walk("split"), (_, _, []))[2]:
    filename = f"split/{filename}"
    if filename.endswith("_filtered.csv"):
        l.append(pd.read_csv(filename, index_col=0))
df = pd.concat(l)


In [ ]:
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)

df.to_csv("split/all_irrelevant_filtered.csv")

Get all the irrelevant CSVs

In [ ]:
l = []
for filename in next(os.walk("split"), (_, _, []))[2]:
    filename = f"split/{filename}"
    if filename.endswith("irrelevant_filtered.csv"):
        l.append(pd.read_csv(filename, index_col=0))
df = pd.concat(l)

if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)

df.to_csv("split/all_irrelevant_filtered.csv")

seconds = time.time() - t0
duration = time.strftime("(%H:%M:%S)",time.gmtime(seconds))
print('Time to complete:',duration)

